In [ ]:
"""
Author: Bryan Bo Cao
Email: boca7588@colorado.edu or bo.cao-1@colorado.edu
Github Repo: https://github.com/BryanBo-Cao/neuralnets-deeplearning
Reference:
    http://www.deeplearning.net/tutorial/lstm.html#lstm
    https://github.com/llSourcell/LSTM_Networks/blob/master/LSTM%20Demo.ipynb
    https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py
    Recurrent Neural Network.
    A Recurrent Neural Network (LSTM) implementation example using TensorFlow library.
    This example is using the MNIST database of handwritten digits (http://yann.lecun.com/exdb/mnist/)
    Links:
    [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf)
    [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).
    Author: Aymeric Damien
    Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import random
import copy
import matplotlib.pyplot as plt
from importlib import import_module

H = 5
N = 2

#Reference: Denis
def generate_parity_sequences(N, count):
    """
    Generate :count: sequences of length :N:.
    If odd # of 1's -> output 1
    else -> output 0
    """
    xor = lambda x: 1 if (x % 2 == 1) else 0
    sequences = np.random.choice([0, 1], size=[count, N], replace=True)
    counts = np.count_nonzero(sequences == 1, axis=1)
    # xor each sequence, expand dimensions by 1 to match sequences shape
    y = np.expand_dims(np.array([xor(x) for x in counts]), axis=1)

    # In case if you wanted to have the answer just appended at the end of the sequence:
    #     # append the answer at the end of each sequence
    #     seq_plus_y = np.concatenate([sequences, y], axis=1)
    #     print(sequences.shape, y.shape, seq_plus_y.shape)
    #     return seq_plus_y
    return np.expand_dims(sequences, axis=2), y

#Reference: Modified from Denis by Bo Cao
def generate_parity_sequences(N, count):
    """
    Generate :count: sequences of length :N:.
    If odd # of 1's -> output 1
    else -> output 0
    """
    xor = lambda x: 1 if (x % 2 == 1) else 0
    sequences = np.random.choice([0, 1], size=[count, N], replace=True)
    counts = np.count_nonzero(sequences == 1, axis=1)
    # xor each sequence, expand dimensions by 1 to match sequences shape
    y = np.expand_dims(np.array([xor(x) for x in counts]), axis=1)

    # In case if you wanted to have the answer just appended at the end of the sequence:
    #     # append the answer at the end of each sequence
    #     seq_plus_y = np.concatenate([sequences, y], axis=1)
    #     print(sequences.shape, y.shape, seq_plus_y.shape)
    #     return seq_plus_y
    
    new_y = []
    for i in range(len(y)):
        new_yy = []
        if y[i] == 0:
            new_yy.append(0)
            new_yy.append(1)
        else:
            new_yy.append(1)
            new_yy.append(0)
        new_y.append(new_yy)

    return np.expand_dims(sequences, axis=2), new_y

In [ ]:
# Training Parameters
learning_rate = 0.000275
batch_size = 128
display_step = batch_size * 100

#batch_steps = 10000 / batch_size
epochs = 1500

# Network Parameters
num_input = 1 # 
timesteps = N # timesteps
num_hidden = H # hidden layer num of features
num_classes = 2 # 0 or 1

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a RNN cell with tensorflow
    rnn_cell = rnn.BasicRNNCell(num_hidden)

    # Get RNN cell output
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits = RNN(X, weights, biases)
#prediction = tf.nn.softmax(logits)
prediction = tf.tanh(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

test_accuracies_10replications = []
losses_1st_replication = [] #epoch as unit
test_accuracies_1st_replication = [] #epoch as unit
train_accuracies_1st_replication = [] #epoch as unit

# Start training
with tf.Session() as sess:
    
    # Run 10 replications
    for replication in range(10):
        
        print("Replication: %d: " % replication)
        
        # Initialize random weights
        train_data = generate_parity_sequences(N, 10000)
        train_data_x = train_data[0]
        train_data_y = train_data[1]
        test_data = generate_parity_sequences(N, 10000)
        test_data_x = test_data[0]
        test_data_y = test_data[1]
        
        # Run the initializer
        sess.run(init)
    
        for epoch in range(epochs):
            batch_index = 0
            while batch_index < 10000:

                train_data_batch_x = []
                train_data_batch_y = []
                if batch_index + batch_size < 10000: 
                    train_data_batch_x = train_data_x[batch_index : batch_index + batch_size]
                    train_data_batch_y = train_data_y[batch_index : batch_index + batch_size]
                else:
                    train_data_batch_x = train_data_x[batch_index : ]
                    train_data_batch_y = train_data_y[batch_index : ]

                #batch_x, batch_y = mnist.train.next_batch(batch_size)
                # Reshape data to get 28 seq of 28 elements
                #batch_x = batch_x.reshape((batch_size, timesteps, num_input))
                #train_data_x = train_data_x.reshape((10000, timesteps, num_input))
                #print("train_data_batch_x.shape:  " , train_data_batch_x.shape)
                if batch_index + batch_size < 10000: 
                    train_data_batch_x = train_data_batch_x.reshape((batch_size, timesteps, num_input))
                else:
                    train_data_batch_x = train_data_batch_x.reshape((10000 % batch_size, timesteps, num_input))
                # Run optimization op (backprop)
                #sess.run(train_op, feed_dict={X: train_data_x, 
                 #                             Y: train_data_y})
                sess.run(train_op, feed_dict={X: train_data_batch_x, 
                                              Y: train_data_batch_y})

                batch_index += batch_size

            if replication == 0:
                loss, train_accuracy = sess.run([loss_op, accuracy], feed_dict={X: train_data_x, Y: train_data_y})
                test_accuracy = sess.run(accuracy, feed_dict={X: test_data_x, Y: test_data_y})
                losses_1st_replication.append(loss)
                train_accuracies_1st_replication.append(train_accuracy)
                test_accuracies_1st_replication.append(test_accuracy)
            
            if epoch % 10 == 0:
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: train_data_x,
                                                                         Y: train_data_y})
                print("Epoch: " + str(epoch) + \
                          ", Loss= " + \
                          "{:.4f}".format(loss) + ", Training Accuracy= " + \
                          "{:.3f}".format(acc))

        print("Optimization Finished!")

        test_data_x = test_data_x.reshape((-1, timesteps, num_input))
        test_data_y = test_data_y
        test_accuracy = sess.run(accuracy, feed_dict={X: test_data_x, Y: test_data_y})
        test_accuracies_10replications.append(test_accuracy)
        print("Testing Accuracy:", test_accuracy)

Replication: 0: 
Epoch: 0, Loss= 0.8956, Training Accuracy= 0.497
Epoch: 10, Loss= 0.8094, Training Accuracy= 0.497
Epoch: 20, Loss= 0.7650, Training Accuracy= 0.497
Epoch: 30, Loss= 0.7369, Training Accuracy= 0.497
Epoch: 40, Loss= 0.7197, Training Accuracy= 0.497
Epoch: 50, Loss= 0.7096, Training Accuracy= 0.497
Epoch: 60, Loss= 0.7036, Training Accuracy= 0.497
Epoch: 70, Loss= 0.7000, Training Accuracy= 0.497
Epoch: 80, Loss= 0.6977, Training Accuracy= 0.497
Epoch: 90, Loss= 0.6962, Training Accuracy= 0.251
Epoch: 100, Loss= 0.6951, Training Accuracy= 0.251
Epoch: 110, Loss= 0.6943, Training Accuracy= 0.502
Epoch: 120, Loss= 0.6936, Training Accuracy= 0.502
Epoch: 130, Loss= 0.6930, Training Accuracy= 0.502
Epoch: 140, Loss= 0.6925, Training Accuracy= 0.502
Epoch: 150, Loss= 0.6919, Training Accuracy= 0.502
Epoch: 160, Loss= 0.6914, Training Accuracy= 0.502
Epoch: 170, Loss= 0.6909, Training Accuracy= 0.502
Epoch: 180, Loss= 0.6904, Training Accuracy= 0.502
Epoch: 190, Loss= 0.6899,

Epoch: 90, Loss= 0.7278, Training Accuracy= 0.498
Epoch: 100, Loss= 0.7210, Training Accuracy= 0.498
Epoch: 110, Loss= 0.7155, Training Accuracy= 0.498
Epoch: 120, Loss= 0.7109, Training Accuracy= 0.498
Epoch: 130, Loss= 0.7070, Training Accuracy= 0.498
Epoch: 140, Loss= 0.7037, Training Accuracy= 0.498
Epoch: 150, Loss= 0.7009, Training Accuracy= 0.498
Epoch: 160, Loss= 0.6985, Training Accuracy= 0.498
Epoch: 170, Loss= 0.6964, Training Accuracy= 0.498
Epoch: 180, Loss= 0.6945, Training Accuracy= 0.498
Epoch: 190, Loss= 0.6929, Training Accuracy= 0.498
Epoch: 200, Loss= 0.6914, Training Accuracy= 0.498
Epoch: 210, Loss= 0.6900, Training Accuracy= 0.498
Epoch: 220, Loss= 0.6887, Training Accuracy= 0.498
Epoch: 230, Loss= 0.6875, Training Accuracy= 0.498
Epoch: 240, Loss= 0.6864, Training Accuracy= 0.498
Epoch: 250, Loss= 0.6853, Training Accuracy= 0.498
Epoch: 260, Loss= 0.6843, Training Accuracy= 0.498
Epoch: 270, Loss= 0.6833, Training Accuracy= 0.498
Epoch: 280, Loss= 0.6823, Traini

Epoch: 180, Loss= 0.6852, Training Accuracy= 0.504
Epoch: 190, Loss= 0.6843, Training Accuracy= 0.504
Epoch: 200, Loss= 0.6835, Training Accuracy= 0.504
Epoch: 210, Loss= 0.6825, Training Accuracy= 0.504
Epoch: 220, Loss= 0.6815, Training Accuracy= 0.504
Epoch: 230, Loss= 0.6805, Training Accuracy= 0.504
Epoch: 240, Loss= 0.6793, Training Accuracy= 0.504
Epoch: 250, Loss= 0.6780, Training Accuracy= 0.504
Epoch: 260, Loss= 0.6767, Training Accuracy= 0.504
Epoch: 270, Loss= 0.6752, Training Accuracy= 0.504
Epoch: 280, Loss= 0.6735, Training Accuracy= 0.504
Epoch: 290, Loss= 0.6716, Training Accuracy= 0.504
Epoch: 300, Loss= 0.6696, Training Accuracy= 0.504
Epoch: 310, Loss= 0.6673, Training Accuracy= 0.504
Epoch: 320, Loss= 0.6647, Training Accuracy= 0.504
Epoch: 330, Loss= 0.6619, Training Accuracy= 0.504
Epoch: 340, Loss= 0.6586, Training Accuracy= 0.504
Epoch: 350, Loss= 0.6550, Training Accuracy= 0.752
Epoch: 360, Loss= 0.6508, Training Accuracy= 0.752
Epoch: 370, Loss= 0.6461, Train

Epoch: 270, Loss= 0.5334, Training Accuracy= 0.749
Epoch: 280, Loss= 0.5284, Training Accuracy= 0.749
Epoch: 290, Loss= 0.5234, Training Accuracy= 0.749
Epoch: 300, Loss= 0.5182, Training Accuracy= 0.749
Epoch: 310, Loss= 0.5130, Training Accuracy= 0.749
Epoch: 320, Loss= 0.5077, Training Accuracy= 0.749
Epoch: 330, Loss= 0.5023, Training Accuracy= 0.749
Epoch: 340, Loss= 0.4969, Training Accuracy= 0.749
Epoch: 350, Loss= 0.4914, Training Accuracy= 0.749
Epoch: 360, Loss= 0.4858, Training Accuracy= 0.749
Epoch: 370, Loss= 0.4802, Training Accuracy= 0.749
Epoch: 380, Loss= 0.4745, Training Accuracy= 0.749
Epoch: 390, Loss= 0.4688, Training Accuracy= 0.749
Epoch: 400, Loss= 0.4630, Training Accuracy= 0.749
Epoch: 410, Loss= 0.4571, Training Accuracy= 0.749
Epoch: 420, Loss= 0.4512, Training Accuracy= 0.749
Epoch: 430, Loss= 0.4453, Training Accuracy= 1.000
Epoch: 440, Loss= 0.4393, Training Accuracy= 1.000
Epoch: 450, Loss= 0.4333, Training Accuracy= 1.000
Epoch: 460, Loss= 0.4273, Train

Epoch: 360, Loss= 0.6538, Training Accuracy= 1.000
Epoch: 370, Loss= 0.6516, Training Accuracy= 1.000
Epoch: 380, Loss= 0.6494, Training Accuracy= 1.000
Epoch: 390, Loss= 0.6470, Training Accuracy= 1.000
Epoch: 400, Loss= 0.6446, Training Accuracy= 1.000
Epoch: 410, Loss= 0.6421, Training Accuracy= 1.000
Epoch: 420, Loss= 0.6394, Training Accuracy= 1.000
Epoch: 430, Loss= 0.6367, Training Accuracy= 1.000
Epoch: 440, Loss= 0.6338, Training Accuracy= 1.000
Epoch: 450, Loss= 0.6308, Training Accuracy= 1.000
Epoch: 460, Loss= 0.6277, Training Accuracy= 1.000
Epoch: 470, Loss= 0.6244, Training Accuracy= 1.000
Epoch: 480, Loss= 0.6211, Training Accuracy= 1.000
Epoch: 490, Loss= 0.6175, Training Accuracy= 1.000
Epoch: 500, Loss= 0.6139, Training Accuracy= 1.000
Epoch: 510, Loss= 0.6100, Training Accuracy= 1.000
Epoch: 520, Loss= 0.6060, Training Accuracy= 1.000
Epoch: 530, Loss= 0.6019, Training Accuracy= 1.000
Epoch: 540, Loss= 0.5976, Training Accuracy= 1.000
Epoch: 550, Loss= 0.5931, Train

In [ ]:
# print results
test_accuracies_10replications_std = np.std(test_accuracies_10replications, axis=0)
test_accuracies_10replications_std_mean = test_accuracies_10replications_std / np.square(10)
print("test_accuracies_10replications: ", test_accuracies_10replications)
print("mean of test_accuracies_10replications: ", np.mean(test_accuracies_10replications))
print("standard deviation of test_accuracies_10replications_std_mean: ", test_accuracies_10replications_std_mean)

In [ ]:
losses_1st_replication
plt.plot(losses_1st_replication, color='green', linewidth=5)
plt.plot(train_accuracies_1st_replication, color='blue', linewidth=7)
plt.plot(test_accuracies_1st_replication, color='red', linewidth=3)
plt.xlim(0, epochs)
plt.xlabel("epoch")
plt.ylim(0, 1.2)
plt.ylabel("train acc in blue, test acc in red, loss in green")
plt.title("1st replication")
plt.show()